# Space Invaders v01 Model Training

Install dependancies

In [ ]:
!pip install -I gym==0.17.3
!pip install keras
!pip install keras-rl2
!pip install atari-py
!pip install autorom


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.6 MB/s 
     |████████████████████████████████| 15.7 MB 34.6 MB/s 
     |████████████████████████████████| 1.0 MB 45.5 MB/s 
     |████████████████████████████████| 829 kB 45.7 MB/s 
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654653 sha256=3df5948dba45975d0ea9a293c869d09a7efdf3e6c5cad9fb700080b22be32003
  Stored in directory: /root/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=0724ed88e54df292ec23bf455ea1de5e2d865d8eade4168b2cf1c111b7a2ef15
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built gym future


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 725 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Install Atari ROMS

In [ ]:
!mkdir sample_data/ROMS
!AutoROM --install-dir sample_data/ROMS --accept-license
!python -m atari_py.import_roms sample_data/ROMS

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/content/sample_data/ROMS

Existing ROMs will be overwritten.
Installed sample_data/ROMS/adventure.bin
Installed sample_data/ROMS/air_raid.bin
Installed sample_data/ROMS/alien.bin
Installed sample_data/ROMS/amidar.bin
Installed sample_data/ROMS/assault.bin
Installed sample_data/ROMS/asterix.bin
Installed sample_data/ROMS/asteroids.bin
Installed sample_data/ROMS/atlantis.bin
Installed sample_data/ROMS/atlantis2.bin
Installed sample_data/ROMS/backgammon.bin
Installed sample_data/ROMS/bank_heist.bin
Installed sample_data/ROMS/basic_math.bin
Installed sample_data/ROMS/battle_zone.bin
Installed sample_data/ROMS/beam_rider.bin
Installed sample_data/ROMS/berzerk.bin
Installed sample_data/ROMS/blackjack.bin
Installed sample_data/ROMS/bowling.bin
Installed sample_data/ROMS/boxing.bin
Installed sample_data/ROMS/breakout.bin
Installed sample_data/ROMS/carnival.bin
Installed sample_data/ROMS/casino.bin
Installed sample_data/ROM

In [ ]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent  # pip install keras-rl2
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy




In [ ]:
env = gym.make("SpaceInvaders-v0")


def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


height, width, channels = env.observation_space.shape
actions = env.action_space.n

model = build_model(height, width, channels, actions)


def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=1, value_test=2,
                                  nb_steps=10000)
    memory = SequentialMemory(limit=2000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg',
                   nb_actions=actions, nb_steps_warmup=1000)
    return dqn


dqn = build_agent(model, actions)

dqn.compile(Adam(lr=0.0001))

dqn.fit(env, nb_steps=20000, visualize=False, verbose=1)
dqn.save_weights('sample_data/SpaceInvaders-v0.h5f', overwrite=True)
print('Saved model to sample_data/SpaceInvaders-v0.h5f')




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 20000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 18144s 2s/step - reward: 0.2700
12 episodes - episode_reward: 223.750 [95.000, 515.000] - loss: 2.869 - mean_q: 14.237 - mean_eps: 1.000 - ale.lives: 1.976

Interval 2 (10000 steps performed)
 2568/10000 [======>.......................] - ETA: 4:13:21 - reward: 0.3018

# New section

# New section